In [15]:
pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


**OPENAI API - INFERENCE - ON FORMAT PRESERVED OCR TEXT**

In [7]:
import openai

# Your OpenAI API key
openai.api_key = ''

# Define the OCR extracted text
ocr_text = """
                 Standard Written Order (SWO)
                           PRESCRIPTION
1. Beneficiary I       NAME
   Patient Name:     HERE        William Thomson
2. Item for           ITEM
   Order:             HERE        Metformin 500 mg abeslncpescQeA42-9
3. Diagnosis/Condition
   Relating to the Need
   for the Item Ordered:              ICD-10 CODE          DIAGNOSIS
                                    k35          89    acute appendicitis without perforation
         ADD ALL                  R10         31     Abdominal pain
         CODES AND
         CONDITIONS
         THAT APPLY
4. Length of         LON
   Need:             HERE         1           # of months (99 = lifetime)
5. Physician's        SIGN         Ma/
   Signature:         HERE
                                             (no signature stamps)
                      NPI #
                      HERE
6. Date:              DATE
                      HERE      April 29 2010
    **ALL SECTIONS MUST BE WRITTEN IN BY THE ORDERING PHYSICIAN**
"""
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"""
Extract the following fields from the text:

1. Beneficiary name
2. Description of the item/HCPCS code
3. Duration of use if applicable
4. Quantity to be dispensed, if applicable
5. Date of the order
6. Physician/practitioner signature
7. all diagnosis codes

Text:
{ocr_text}

Fields:
"""}
]

# Use OpenAI API to get the response
# Use OpenAI API to get the response
# Use OpenAI API to get the response
try:
    response = openai.ChatCompletion.create(
      model="gpt-4",  # Use the latest available model
      messages=messages,
      max_tokens=200,
      temperature=0.0
    )

    # Print the extracted fields
    print(response.choices[0].message['content'].strip())

except openai.error.InvalidRequestError as e:
    print(f"Error: {e}")

except Exception as e:
    print(f"An error occurred: {e}")


1. Beneficiary name: William Thomson
2. Description of the item/HCPCS code: Metformin 500 mg abeslncpescQeA42-9
3. Duration of use if applicable: 1 month
4. Quantity to be dispensed, if applicable: Not mentioned in the text
5. Date of the order: April 29, 2010
6. Physician/practitioner signature: Ma/
7. All diagnosis codes: K35.89 (acute appendicitis without perforation), R10.31 (Abdominal pain)


In [1]:
pip install ollama



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


**LLAMA3 INFERENCE - ON FORMAT PRESERVED OCR TEXT**

In [2]:
from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please extract the following fields:\n"
        "1. Beneficiary name\n"
        "2. Description of the item and its HCPCS code\n"
        "3. Duration of use, if applicable\n"
        "4. Quantity to be dispensed, if applicable\n"
        "5. Date of the order\n"
        "6. Physician/practitioner signature\n"
        "7. List all diagnosis codes"
    )
    
    qa_template = PromptTemplate(query)

    # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."

# Define the OCR extracted text
ocr_text = """
Standard Written Order (SWO)
         PRESCRIPTION
1. Beneficiary I NAME
   Patient Name:     HERE        William Thomson
2. Item for           ITEM
   Order:             HERE        Metformin 500 mg abeslncpescQeA42-9
3. Diagnosis/Condition
   Relating to the Need
   for the Item Ordered:              ICD-10 CODE          DIAGNOSIS
                                    k35          89    acute appendicitis without perforation
         ADD ALL                  R10         31     Abdominal pain
         CODES AND
         CONDITIONS
         THAT APPLY
4. Length of         LON
   Need:             HERE         1           # of months (99 = lifetime)
5. Physician's        SIGN         Ma/
   Signature:         HERE
                                             (no signature stamps)
                      NPI #
                      HERE
6. Date:              DATE
                      HERE      April 29 2010
    **ALL SECTIONS MUST BE WRITTEN IN BY THE ORDERING PHYSICIAN**
"""

# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)


/Users/areefa/Desktop/OCR_Models/table_detection_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Response: {'model': 'llama3', 'created_at': '2024-06-24T01:21:33.621432Z', 'message': {'role': 'assistant', 'content': 'Based on the provided context information, I can extract the following fields:\n\n1. Beneficiary name: William Thomson\n2. Description of the item and its HCPCS code: Metformin 500 mg abeslncpescQeA42-9 (Note: This is likely a medication name and its corresponding HCPCS code)\n3. Duration of use, if applicable: # of months (99 = lifetime), which means the medication is prescribed for a lifetime duration\n4. Quantity to be dispensed, if applicable: Not specified, as this information is not provided in the SWO\n5. Date of the order: April 29, 2010\n6. Physician/practitioner signature: Ma/ (signature written by hand)\n7. List all diagnosis codes:\n\t* ICD-10 CODE: k35\n\t* Diagnosis: acute appendicitis without perforation\n\t* ADD ALL R10\n\t* ADD CONDITIONS: 31 Abdominal pain'}, 'done_reason': 'stop', 'done': True, 'total_duration': 15317091459, 'load_duration': 7742548

In [7]:
import os

from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, What can be the root cause of this testcase failure\n"
    )
    
    qa_template = PromptTemplate(query)

    # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."

# Define the OCR extracted text
ocr_text = r'''
Case ID: T1025211
reports/screenshots/stage/2024_05_27__14\\admin_stage_console_sharecare_com__2024_05_27_14h18m04s.png
Failing scenarios:
  features/care_console/rewards_activity_templates/prod_tab/verify_next_previous_tab_functionality.feature:6  Login to care console
0 features passed, 1 failed, 0 skipped
0 scenarios passed, 1 failed, 0 skipped, 1 untested
0 steps passed, 1 failed, 1 skipped, 0 undefined, 3 untested
Took 1m7.596s
Feature: To verify whether on clicking the 'Next or Previous' tab icon at the bottom of the list, it is redirecting to the correct page or not.
Scenario: Login to care console
    Given I login to care console ... failed in 67.596s
Traceback (most recent call last):
  File "C:\\Users\\sunil.mekala\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\behave\\model.py", line 1329, in run
    match.run(runner.context)
  File "C:\\Users\\sunil.mekala\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\behave\\matchers.py", line 98, in run
    self.func(context, *args, **kwargs)
  File "features\\steps\\steps_ac_givens.py", line 208, in care_console_login
    .until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[name=loginfmt]"))) \
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\\Users\\sunil.mekala\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\selenium\\webdriver\\support\\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message:
'''

# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)


Response: {'model': 'llama3', 'created_at': '2024-06-24T01:29:34.826627Z', 'message': {'role': 'assistant', 'content': 'A nice debugging challenge!\n\nAfter reviewing the provided information, I\'ll highlight some potential causes for the test case failure:\n\n1. **Element not found**: The error message suggests that Selenium is unable to locate an element with the CSS selector `input[name=loginfmt]` using `EC.visibility_of_element_located`. This might indicate that the element doesn\'t exist or is not visible at the time of execution.\n2. **Timeout issue**: The test case failed after 67.596 seconds, which might imply that the element was not found within the expected timeout period. This could be due to slow network conditions, an overloaded server, or an incorrect locator strategy.\n3. **Login failure**: Although the scenario is labeled as "Login to care console", the error message doesn\'t explicitly mention login-related issues. However, it\'s possible that the login process itself

**LLAMA3 INFERENCE ON UNSTRUCTURED OCR - FORMAT NOT PRESERVED OCR TEXT**

In [14]:
from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please extract the following fields:\n"
        "1. Beneficiary name\n"
        "2. Description of the item and its HCPCS code\n"
        "3. Duration of use, if applicable\n"
        "4. Quantity to be dispensed, if applicable\n"
        "5. Date of the order\n"
        "6. Physician/practitioner signature\n"
        "7. List all diagnosis codes"
    )
    
    qa_template = PromptTemplate(query)

    # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."

# Define the OCR extracted text
ocr_text = """
Standard Written Order (SWO)
PRESCRIPTION
1. Beneficiary I
Patient Name:
NAME
HERE
ITEM
HERE
William Thomson
2. Item for
Order:
Metformin 500 mg abeslncpescQeA42-9
3. Diagnosis/Condition
Relating to the Need
for the Item Ordered:
ICD-10 CODE
DIAGNOSIS
k35
R10
89
31
acute appendicitis without perforation
Abdominal pain
ADD ALL
CODES AND
CONDITIONS
THAT APPLY
4. Length of
Need:
LON
HERE
SIGN
HERE
NPI #
HERE
1
# of months (99 = lifetime)
5. Physician's
Signature:
Ma/
(no signature stamps)
6. Date:
DATE
HERE
April 29 2010
**ALL SECTIONS MUST BE WRITTEN IN BY THE ORDERING PHYSICIAN**
"""

# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)


Response: {'model': 'llama3', 'created_at': '2024-05-15T21:32:44.00162Z', 'message': {'role': 'assistant', 'content': 'Based on the provided Standard Written Order (SWO), I can extract the following fields:\n\n1. Beneficiary name: William Thomson\n2. Description of the item and its HCPCS code: Metformin 500 mg (no specific HCPCS code mentioned)\n3. Duration of use, if applicable: Number of months = 99 (lifetime)\n4. Quantity to be dispensed, if applicable: Not specified\n5. Date of the order: April 29, 2010\n6. Physician/practitioner signature: Ma/ (no signature stamps)\n\nAs for the diagnosis codes, I can extract the following:\n\n* ICD-10 CODE:\n\t+ k35\n\t+ R10\n\t+ 89\n\t+ 31\n* Diagnosis:\n\t+ Acute appendicitis without perforation\n\t+ Abdominal pain\n\nPlease note that there may be additional diagnosis codes or conditions not mentioned in this specific SWO.'}, 'done_reason': 'stop', 'done': True, 'total_duration': 7292443959, 'load_duration': 5475167, 'prompt_eval_count': 298, '

**LLAMA3 INFERENCE - ON FORMAT PRESERVED OCR TEXT - Patient Registration Form**

In [19]:
from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please extract the following fields:\n"
        "1. Beneficiary name\n"
        "2. Patient Account\n"
        "3. Patient Social Security Number\n"
        "4. Birth Date \n"
        "5. Patient Type \n"
        "6. Provisional Diagnosis\n"
        "7. List all diagnosis codes\n"
        "8. Medical record Number\n"
        "9. Policy Number\n"
        "10. Doctor Name\n"
    )
    
    qa_template = PromptTemplate(query)

    # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."

# Define the OCR extracted text
ocr_text = """
                                  Community General Hospital
 Patient Account #           INPATIENT REGISTRATION AND SUMMARY FORM       Medical Record #
   12345                                                                      215043
Patient Name (Las(Firs)(Midile)            Attending Physician Number and Name       Patient Type        Hospital    Admit Date  Admit time
   Brown, John              Jeff T. Moore    97   Inp         Services S 11/12/99 11:10
Patient Address (Strec) (City) (State) (ZipCode)                                           Patient Phone #        Date of Prev. Admit
   27 Cottonwood Ln   Anytown   USA                             123-123-4567
Previous or Maiden Name                 Birth Date     Age Sex Marital Religion      Comments
                             9/10/44   54 M St M             Donor No
Notify in Case of Emergency                       Address             City/State       Phone         Relationship
Patient Social Security Number   Employer Name                       Employer City/State     Guarantor #   Guarantor Name
123-45-6789   Big Company                   Anytown, US            John Brown
Guarantor Address                                                                Guarantor Social Security Number
   Same
Payer                           Policy Number                   Insured's Name          Group Name
   Southern Company           123456789                 John Brown
Financial Class             Admitted By                    Patient Weight                   Discharge Date  Disch. Time
                     AD12                                             11/13/99 11:30
Provisional Diagnosis
   Right Ing Hernia
Principal Diagnosis, Secondary Diagnosis and Complications
                                                                         550, 9/
                                                                         3144
Principal Procedures and Secondary Procedures
                                                                         53.03
                                                           1-12        63.3
                                                           More
                                                                         49530
Consuitations
Dispgsition Home of Case      Swing                                                    Coder     Date
 /                Bed     Supervised Living      Home      Nursing Home
                                        Health                            kp
a Expired        Autopsy      AMA
  Transferred to:                          Icertify that the narrative description ofthe principal and secondary diagnoses and the major procedures
                                    performed are accurate and complete to the best of my knowledge.
                                    Attending Physician                                Date
"""

# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)


Response: {'model': 'llama3', 'created_at': '2024-05-15T21:52:17.396173Z', 'message': {'role': 'assistant', 'content': "Based on the provided information, I can extract the following fields:\n\n1. Beneficiary name: John Brown (also listed as the patient's name)\n2. Patient Account: 12345\n3. Patient Social Security Number: 123-45-6789\n4. Birth Date: 9/10/44\n5. Patient Type: Inpatient Services S\n6. Provisional Diagnosis: Right Ing Hernia\n7. List all diagnosis codes:\n\t* Principal Diagnosis: 550, 9 (no further information)\n\t* Secondary Diagnosis: 3144 (no further information)\n8. Medical record Number: 215043\n9. Policy Number: 123456789\n10. Doctor Name: Jeff T. Moore"}, 'done_reason': 'stop', 'done': True, 'total_duration': 7214847333, 'load_duration': 6367333, 'prompt_eval_count': 548, 'prompt_eval_duration': 2122707000, 'eval_count': 152, 'eval_duration': 5073442000}
Based on the provided information, I can extract the following fields:

1. Beneficiary name: John Brown (also l

**LLAMA3 INFERENCE - NO FORMAT PRESERVED - UNSTRUCTURED OCR TEXT**

In [18]:
from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please extract the following fields:\n"
        "1. Beneficiary name\n"
        "2. Patient Account\n"
        "3. Patient Social Security Number\n"
        "4. Birth Date \n"
        "5. Patient Type \n"
        "6. Provisional Diagnosis\n"
        "7. List all diagnosis codes\n"
        "8. Medical record Number\n"
        "9. Policy Number\n"
    )
    
    qa_template = PromptTemplate(query)

    # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."

# Define the OCR extracted text
ocr_text = """
Community
GeneralHospital
Patient
Account
#
MedicalRecord
#
INPATIENTREGISTRATIONAND
SUMMARYFORM
12345
215043
Name
(Las(Firs)(Midile)
Patient  Attending
PhysicianNumberPatientTypeHospital
andName  AdmitDateAdmittime
Services
Brown,John
Jeff    11/12/99
T. 97  S
Moore Inp  11:10
Address(Strec)
Patient(City)(State)(ZipCode)
ofPrev.Admit
PatientPhoneDate
#
27Cottonwood
LnAnytown
USA  123-123-4567
PreviousMaiden Birth
orName DateAgeMarital
Sex
ReligionComments
St
9/10/44
54M
M  No
Donor
NotifyCaseof
inEmergency
Address
City/State
PhoneRelationship
PatientSocialSecurity
NumberEmployerName  EmployerCity/State
GuarantorName
Guarantor#
123-45-6789
BigCompany
Anytown,US John
Brown
Address
Guarantor
GuarantorSecurityNumber
Social
Same
Payer
PolicyNumber
Insured's
NameGroup
Name
Southern
Company
123456789
John
Brown
FinancialClass By
Admitted
PatientWeight
Date
DischargeTime
Disch.
AD12
11/13/99
11:30
ProvisionalDiagnosis
RightIng
Hernia
PrincipalDiagnosis,Secondary
DiagnosisComplications
and
9/
550,
3144
PrincipalProceduresandSecondaryProcedures
53.03
1-12
63.3
More
49530
Consuitations
Dispgsitionof
Case
CoderDate
Home
Swing
Bed
Living
/  SupervisedHomeNursingHome
kp
Health
a Autopsy
Expired AMA
Transferred
to:  Icertifynarrativeofthe the
thatthedescriptionprincipalsecondarydiagnosesand
and majorprocedures
performed best
accurateandcompletetotheofmyknowledge.
are
Attending
Physician
Date
"""

# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)


Response: {'model': 'llama3', 'created_at': '2024-05-15T21:50:36.68981Z', 'message': {'role': 'assistant', 'content': 'Here are the extracted fields:\n\n1. Beneficiary name: John Brown\n2. Patient Account: #12345\n3. Patient Social Security Number: 123-45-6789\n4. Birth Date: 9/10/44\n5. Patient Type: Inpatient\n6. Provisional Diagnosis: Right Ing Hernia\n7. List of diagnosis codes:\n\t* Principal Diagnosis: 550, 3144\n\t* Secondary Diagnosis: 53.03, 63.3\n8. Medical record Number: #12345 (same as Patient Account)\n9. Policy Number: 123456789'}, 'done_reason': 'stop', 'done': True, 'total_duration': 7601902583, 'load_duration': 1426469541, 'prompt_eval_count': 573, 'prompt_eval_duration': 1984093000, 'eval_count': 129, 'eval_duration': 4182713000}
Here are the extracted fields:

1. Beneficiary name: John Brown
2. Patient Account: #12345
3. Patient Social Security Number: 123-45-6789
4. Birth Date: 9/10/44
5. Patient Type: Inpatient
6. Provisional Diagnosis: Right Ing Hernia
7. List of 

**SLM - SLIM-EXTRACT**

In [3]:
from llama_index.core import PromptTemplate
from ollama import Client

# Host the Ollama on your local machine
client = Client(host='http://localhost:11434')

def extract_fields_from_ocr(ocr_text):
    query = (
        "We have provided context information below. \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "Given this information, please extract the following fields values. Don't give any sentences:\n"
        "1. Time\n"
        "1. B/P\n"
        "1. T\n"
        
    )
    
    qa_template = PromptTemplate(query)
     # Create the text prompt
    prompt = qa_template.format(context_str=ocr_text, query_str='')

    response = client.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': prompt
        },
    ])
    
    # Print the response to understand its structure
    print("Response:", response)
    
    # Extract the content from the response
    try:
        resp_text = response['message']['content']
        return resp_text
    except KeyError as e:
        return f"Unexpected response structure: Key {e} not found."


ocr_text='''
Race                                                                   Time
Number:                                                                In:
(or non participant)
                                                                        Tent
Contact #:                    Medical Treatment Record   location:
                                     Elite          DAge Group
      POC: Transition Tent   On course   Main Medical Tent (Observation a Routinel D Urgent D)
      Arrived from: Race course: OS OB OR OTIOT2 D Finish Line D Other          Finisher D Yes D No
      Last Name Smith            First Name Jane                  Age 54        Sex F
      ChiefComplaint: DExhaustion D Dizziness OSyncope/collapse OMuscle Cramps  Headache
      D Altered Mental Status D Seizures - Nausea D Vomiting D Diarrhea D ABD Cramps  Short ofBreath D Chest
      pain 0 Trauma D Other
      Pre-race weight:                 Post-race weight:              Last urination:
      Allergies: ONKDA                Pertinent Med Hx / Meds:
      HPI:
      Pertinent PE:
      Provisional Diagnosis: OExhaustion/EAC ODehydration Heat cramps OHyperthermia OHypothermia
      DHypoglycemia OHyponatremia OHypokalemia Trauma
      OPulmonary.                 D Cardiac                   DOther
      Treatment: DOral fluids Head down/feet up a Calories/glucose D Warm liquids 0 Blanket D Ice packs D Cold
      water immersion D Stretching a Wound dressing D Blister care
      D Other:                            DIVF                    OMedication
Time: 11:00 am  T:98.6° P: 70 R: 16 BP: 120/80 IVF:        Bag #/amt     Medications:
Time: 12:00p pm  T:100 P: 80 R: 20 BP:130/85 IVF:        Bag #/amt     Medications:
Time: 2:00 pm   T97.8° P: 60 R: 12 BP: 110/70 IVF:        Bag #/amt     Medications:
Time:          T:   P:    R:   BP:      IVF:        Bag #/amt     Medications:
Time:          T:   P:   R:   BP:      IVE:        Bag #/amt     Medications:
      Narrative/Labs:
      Disposition: a Resume Race     Home: via             Accompanied by
        Hospital/EMS             Notified:               Time:       Records sent: D Yes D No
      Condition at Discharge:  Stable  Fair Guarded OCritical
      Follow Up: Lz ED (non emergent transport) J X-ray - Wound care  Personal physician a Other:
                                                                /
      Provider: Print name, Sign/Initial, Title        Provider: Print name, Sign/Initial, Title
'''
# Extract fields from OCR text
extracted_fields = extract_fields_from_ocr(ocr_text)
print(extracted_fields)



Response: {'model': 'llama3', 'created_at': '2024-05-29T07:17:47.477408Z', 'message': {'role': 'assistant', 'content': 'Here are the extracted field values:\n\n**Time**\n1. 11:00 am\n2. 12:00 pm\n3. 2:00 pm\n\n**B/P**\n1. BP: 120/80\n2. BP: 130/85\n3. BP: 110/70\n\n**T**\n1. T: 98.6°\n2. T: 100\n3. T: 97.8°'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3395285792, 'load_duration': 7928584, 'prompt_eval_count': 17, 'prompt_eval_duration': 279068000, 'eval_count': 95, 'eval_duration': 3095310000}
Here are the extracted field values:

**Time**
1. 11:00 am
2. 12:00 pm
3. 2:00 pm

**B/P**
1. BP: 120/80
2. BP: 130/85
3. BP: 110/70

**T**
1. T: 98.6°
2. T: 100
3. T: 97.8°
